In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from tensorflow.keras.layers import Input,Add,Flatten, Dense, LayerNormalization, MultiHeadAttention, Dropout, GlobalAveragePooling1D,LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import torch

In [ ]:
data_path = 'golddemandQ.csv'  # Update this to the correct path where IBM.csv is located
data = pd.read_csv(data_path)
data=data.iloc[:,0:2]

In [ ]:
import warnings

warnings.simplefilter("ignore", category=DeprecationWarning)
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.activations import get
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
# Preprocess data
# Set random seed for reproducibility
with pd.ExcelWriter('transformer-demand-mult_sheets.xlsx') as writer:
    for r in range(10):  # 循环运行10次


        data_path = 'golddemandQ.csv'  # Update this to the correct path where IBM.csv is located
        data = pd.read_csv(data_path)
        data=data.iloc[:,[0,2]]

        np.random.seed(42)
        torch.manual_seed(42)
        tf.random.set_seed(42)

        import pandas as pd
        import numpy as np
        from sklearn.preprocessing import MinMaxScaler
        from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, Dropout, LayerNormalization, Add, Flatten
        from tensorflow.keras.models import Model
        import matplotlib.pyplot as plt
        from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error



        # Display first few rows of the dataset
        print(data.head())

        # Ensure the 'Date' column is datetime type and set it as index if necessary
        # data['time'] = pd.to_datetime(data['time'])
        data.set_index('time', inplace=True)

        # Use 'Close' price for prediction
        data = data[['closed']]



        def create_dataset(data, time_step=2):
            X, y = [], []
            for i in range(len(data) - time_step):
                X.append(data[i:i + time_step])
                y.append(data[i + time_step])
            return np.array(X), np.array(y)

        # Create dataset
        X, y = create_dataset(data[['closed']].values, time_step)

        # Split data into training and testing sets (80% for training, 20% for testing)
        train_size = int(len(X) * 0.8)
        X_train0, y_train0 = X[:train_size], y[:train_size]
        X_test0, y_test0 = X[train_size:], y[train_size:]

        # Standardize the training data
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train= scaler.fit_transform(X_train0.reshape(-1, 1)).reshape(X_train0.shape)
        y_train = scaler.fit_transform(y_train0.reshape(-1, 1)).reshape(-1)

        # Apply the same scaling to the test data
        X_test = scaler.transform(X_test0.reshape(-1, 1)).reshape(X_test0.shape)
        y_test = scaler.transform(y_test0.reshape(-1, 1)).reshape(-1)
        # Build the Transformer model
        def build_transformer_model(input_shape):
            inputs = Input(shape=input_shape)

            # Transformer encoder layer
            attention = MultiHeadAttention(num_heads=8, key_dim=32)(inputs, inputs)
            attention = Dropout(0.2)(attention)
            attention = LayerNormalization(epsilon=1e-6)(attention)

            # Add residual connection
            x = Add()([inputs, attention])

            # Dense layers
            x = Flatten()(x)
            x = Dense(128, activation='relu')(x)
            x = Dropout(0.3)(x)
            x = Dense(64, activation='relu')(x)
            outputs = Dense(1)(x)

            model = Model(inputs=inputs, outputs=outputs)
            return model

        # Initialize and compile the model
        input_shape = (X_train.shape[1], X_train.shape[2])
        transformer_model = build_transformer_model(input_shape)
        transformer_model.compile(optimizer='adam', loss='mean_squared_error')

        # Train the model
        history = transformer_model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=50,
            batch_size=1,
            verbose=1
        )

        # Predict on the test set
        transformer_predictions = transformer_model.predict(X_test)
        transformer_predictions = scaler.inverse_transform(transformer_predictions)
        y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Evaluate the model
        transformer_r2 = r2_score(y_test_actual, transformer_predictions)
        transformer_rmse = np.sqrt(mean_squared_error(y_test_actual, transformer_predictions))
        transformer_mae = mean_absolute_error(y_test_actual, transformer_predictions)

        # Evaluate the model using MAE, RMSE, MAPE, and Directional Accuracy
        def calculate_mape(y_true, y_pred):
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        def calculate_directional_accuracy(y_true, y_pred):
            correct = np.sum(np.sign(y_true) == np.sign(y_pred))
            return correct / len(y_true)

        transformer_mape = calculate_mape(y_test_actual, transformer_predictions)
        direction_test = calculate_directional_accuracy(y_test_actual, transformer_predictions)

        # Print evaluation metrics
        print(f"Transformer R² Value: {transformer_r2:.4f}")
        print(f"Transformer RMSE: {transformer_rmse:.4f}")
        print(f"Transformer MAE: {transformer_mae:.4f}")
        print(f"Transformer mape: {transformer_mape:.4f}")


        metrics1 = {
                'RMSE': transformer_rmse,
                # 'MSE': mse,
                'MAE': transformer_mae,
                'MAPE':transformer_mape,
                # 'Directional Accuracy': direction_accuracy,
                # 'actual_price':y_test_actual,
                # 'predictions': transformer_predictions
            }
        # print(metrics1)
        metrics2={'actual_price':np.squeeze(y_test_actual),
                'predictions': np.squeeze(transformer_predictions)}
     
        # df_metrics = pd.DataFrame([metrics],index=[0])
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        df_metrics1 = pd.DataFrame(metrics1,index=[0])
        df_metrics2 = pd.DataFrame(metrics2)
        df_metrics=pd.concat([df_metrics1,df_metrics2])
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}')

In [ ]:
# MIMO

import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor

# Create a random dataset
rng = np.random.RandomState(1)
data_path = 'golddemandQ.csv'  # Update this to the correct path where IBM.csv is located
data = pd.read_csv(data_path)
data=data.iloc[:,:]

data.set_index('time', inplace=True)

data_close = data[['closep','closed']].values
split = int(len(data_close) * 0.8)
data_close_train, data_close_test = data_close[:split], data_close[split:]

def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

def create_sequencestest(data, sequence_length, steps,price_data_test,split):
    xs, ys = [], []
    for i in range(len(price_data_test)):
        x = data[(split-sequence_length)+i:split+i]
        y = data[split+i:split+i+ steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 2  # Updated sequence length for your use case
steps = 1  # Number of future steps to predict



X_train, y_train = create_sequences(data_close_train, sequence_length, steps)
X_test, y_test = create_sequencestest(data_close, sequence_length, steps,price_data_test,split)

X_train=X_train.reshape(-1,2*sequence_length)
y_train = y_train.reshape(-1,1*sequence_length)
X_test=X_test.reshape(-1,2*sequence_length)
y_test = y_test.reshape(-1,1*sequence_length)

max_depth = 50
# regr_multirf = MultiOutputRegressor(BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100))
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=10,
                                                          max_depth=max_depth,
                                                          random_state=0))
regr_multirf.fit(X_train, y_train)

regr_rf = RandomForestRegressor(n_estimators=10, max_depth=max_depth,
                                random_state=2)
regr_rf.fit(X_train, y_train)

# Predict on new data
y_multirf = regr_multirf.predict(X_test)
y_rf = regr_rf.predict(X_test)

mae = mean_absolute_error(y_test[:,0], y_multirf[:,0])
rmse = np.sqrt(mean_squared_error(y_test[:,0], y_multirf[:,0]))

def calculate_mape(y_true, y_pred):
    return np.mean(np.abs(((y_true - y_pred) / y_true))) * 100

mape = calculate_mape(y_test[:,0], y_multirf[:,0])


print('''first-variable-MIMOresults''')
print(f'MAE: {np.mean(mae)}')
print(f'RMSE: {np.mean(rmse)}')
print(f'MAPE: {mape}%')
mae2 = mean_absolute_error(y_test[:,1], y_multirf[:,1])
rmse2 = np.sqrt(mean_squared_error(y_test[:,1], y_multirf[:,1]))
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs(((y_true - y_pred) / y_true))) * 100

mape2 = calculate_mape(y_test[:,1], y_multirf[:,1])


print('''second-variable-MIMOresults''')
print(f'MAE: {np.mean(mae2)}')
print(f'RMSE: {np.mean(rmse2)}')
print(f'MAPE: {mape2}%')


LSTM预测

In [ ]:
#LSTM
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
with pd.ExcelWriter('LSTM-demand-mult_sheets.xlsx') as writer:
    for r in range(10):  
        # Set random seed for reproducibility
        np.random.seed(42)

        # Load the data
        data_path = 'golddemandQ.csv'  # Update this to the correct path where IBM.csv is located
        data = pd.read_csv(data_path)
        data=data.iloc[:,[0,2]]

        def create_dataset(data, time_step=2):
            X, y = [], []
            for i in range(len(data) - time_step):
                X.append(data[i:i + time_step])
                y.append(data[i + time_step])
            return np.array(X), np.array(y)

        # Create dataset
        X, y = create_dataset(data[['closed']].values, time_step)

        # Split data into training and testing sets (80% for training, 20% for testing)
        train_size = int(len(X) * 0.8)
        X_train0, y_train0 = X[:train_size], y[:train_size]
        X_test0, y_test0 = X[train_size:], y[train_size:]

        # Standardize the training data
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train= scaler.fit_transform(X_train0.reshape(-1, 1)).reshape(X_train0.shape)
        y_train = scaler.fit_transform(y_train0.reshape(-1, 1)).reshape(-1)

        # Apply the same scaling to the test data
        X_test = scaler.transform(X_test0.reshape(-1, 1)).reshape(X_test0.shape)
        y_test = scaler.transform(y_test0.reshape(-1, 1)).reshape(-1)

        # Build the LSTM model
        model = Sequential()
        model.add(LSTM(128, input_shape=(sequence_length, 1)))
        model.add(Dropout(0.2))
        model.add(Dense(steps))  # The output layer should have 'steps' units if predicting multiple steps

        # Compile the model
        model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

        # Train the model
        model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1, validation_data=(X_test, y_test))

        # Predicting
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Inverse transform the predictions back to original scale
        train_predict = scaler.inverse_transform(train_predict)
        test_predict = scaler.inverse_transform(test_predict)
        y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
        y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Evaluate the model using MAE, RMSE, MAPE, and Directional Accuracy
        def calculate_mape(y_true, y_pred):
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        def calculate_directional_accuracy(y_true, y_pred):
            correct = np.sum(np.sign(y_true) == np.sign(y_pred))
            return correct / len(y_true)

        mae_train = mean_absolute_error(y_train_inv, train_predict)
        rmse_train = np.sqrt(mean_squared_error(y_train_inv, train_predict))
        mape_train = calculate_mape(y_train_inv, train_predict)
        direction_train = calculate_directional_accuracy(y_train_inv, train_predict)

        mae_test = mean_absolute_error(y_test_inv, test_predict)
        rmse_test = np.sqrt(mean_squared_error(y_test_inv, test_predict))
        mape_test = calculate_mape(y_test_inv, test_predict)
        direction_test = calculate_directional_accuracy(y_test_inv, test_predict)

        print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}, MAPE: {mape_train}%, Directional Accuracy: {direction_train}")
        print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}, MAPE: {mape_test}%, Directional Accuracy: {direction_test}")

        metrics1 = {
                'RMSE': rmse_test,
                # 'MSE': mse,
                'MAE': mae_test,
                'MAPE':mape_test,
                # 'Directional Accuracy': direction_accuracy,
                # 'actual_price':y_test_actual,
                # 'predictions': transformer_predictions
            }
        # print(metrics1)
        metrics2={'actual_price':np.squeeze(y_test_inv),
                'predictions': np.squeeze(test_predict)}
   
        # df_metrics = pd.DataFrame([metrics],index=[0])
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        df_metrics1 = pd.DataFrame(metrics1,index=[0])
        df_metrics2 = pd.DataFrame(metrics2)
        df_metrics=pd.concat([df_metrics1,df_metrics2])
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}')

CNN-LSTM预测

In [ ]:
#CNN-LSTM
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras.optimizers import Adam
with pd.ExcelWriter('CNNLSTM-demand-mult_sheets.xlsx') as writer:
    for r in range(1):  # 
        # Set random seed for reproduciility
        np.random.seed(42)

        
        # Set random seed for reproducibility
        np.random.seed(42)

        # Load the data from a CSV file
        data_path = 'golddemandQ.csv'  # Update this to the correct path where IBM.csv is located
        data = pd.read_csv(data_path)
        data=data.iloc[:,0:2]
        data.set_index('time', inplace=True)

        # Use 'Close' price for prediction
        data = data[['closep']]

        def create_dataset(data, time_step=4):
            X, y = [], []
            for i in range(len(data) - time_step):
                X.append(data[i:i + time_step])
                y.append(data[i + time_step])
            return np.array(X), np.array(y)

        # Create dataset
        X, y = create_dataset(data[['closep']].values, time_step=4)

        # Split data into training and testing sets (80% for training, 20% for testing)
        train_size = int(len(X) * 0.8)
        X_train0, y_train0 = X[:train_size], y[:train_size]
        X_test0, y_test0 = X[train_size:], y[train_size:]

        # Standardize the training data
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train= scaler.fit_transform(X_train0.reshape(-1, 1)).reshape(X_train0.shape)
        y_train = scaler.fit_transform(y_train0.reshape(-1, 1)).reshape(-1)

        # # Apply the same scaling to the test data
        X_test = scaler.transform(X_test0.reshape(-1, 1)).reshape(X_test0.shape)
        y_test = scaler.transform(y_test0.reshape(-1, 1)).reshape(-1)
        subsequences = 1
        timesteps = X_train.shape[1] // subsequences
        X_train = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
        X_test = X_test.reshape((X_test.shape[0], subsequences, timesteps, 1))

        


        # Define the CNN-LSTM model
        model = Sequential()

        # CNN layers within TimeDistributed wrapper to process each subsequence
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None, timesteps, 1)))
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model.add(TimeDistributed(Flatten()))

        # LSTM layers
        model.add(LSTM(128, return_sequences=False))
        model.add(Dropout(0.2))

        # Final dense layers
        # model.add(Dense(64, activation='relu'))
        model.add(Dense(steps))  # Adjust 'steps' if you're predicting more than one future step

        # Compile the model
        model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

        # Train the model
        model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1, validation_data=(X_train, y_train))

        # Predictions
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Inverse transform the predictions back to original scale
        train_predict = scaler.inverse_transform(train_predict)
        test_predict = scaler.inverse_transform(test_predict)
        y_train = scaler.inverse_transform(y_train.reshape(-1,1))
        y_test = scaler.inverse_transform(y_test.reshape(-1,1))

        def calculate_mape(y_true, y_pred):
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        def calculate_directional_accuracy(y_true, y_pred):
            correct = np.sum(np.sign(y_true) == np.sign(y_pred))
            return correct / len(y_true)



        # Evaluate the model using MAE and RMSE
        mae_train = mean_absolute_error(y_train.reshape(-1,1), train_predict)
        rmse_train = np.sqrt(mean_squared_error(y_train.reshape(-1,1),train_predict))
        mape_train = calculate_mape(y_train.reshape(-1,1), train_predict)
        direction_train = calculate_directional_accuracy(y_train.reshape(-1,1), train_predict)

        mae_test = mean_absolute_error(y_test0.reshape(-1,1), test_predict)
        rmse_test = np.sqrt(mean_squared_error(y_test0.reshape(-1,1), test_predict))
        mape_test = calculate_mape(y_test0.reshape(-1,1), test_predict)
        direction_test = calculate_directional_accuracy(y_test0.reshape(-1,1), test_predict)

        print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}, MAPE: {mape_train}%, Directional Accuracy: {direction_train}")
        print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}, MAPE: {mape_test}%, Directional Accuracy: {direction_test}")
        metrics1 = {
                'RMSE': rmse_test,
                # 'MSE': mse,
                'MAE': mae_test,
                'MAPE':mape_test}
                # 'Directional Accuracy': direction_accuracy,
                # 'actual_price':y_test_actual,
                # 'predictions': transformer_predictions
            
        # print(metrics1)
        metrics2={'actual_price':np.squeeze(y_test0),
                'predictions': np.squeeze(test_predict)}
        # df_metrics = pd.DataFrame([metrics],index=[0])
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        df_metrics1 = pd.DataFrame(metrics1,index=[0])
        df_metrics2 = pd.DataFrame(metrics2)
        df_metrics=pd.concat([df_metrics1,df_metrics2])
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}')

SVR预测

In [ ]:
#SVR
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr

# Set random seed for reproducibility
np.random.seed(42)

data_path = 'golddemandQ.csv'  # Update this to the correct path where IBM.csv is located
data = pd.read_csv(data_path)
data=data.iloc[:,0:2]
data.set_index('time', inplace=True)


def create_dataset(data, time_step=2):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

# Create dataset
X, y = create_dataset(data[['closep']].values, time_step)

# Split data into training and testing sets (80% for training, 20% for testing)
train_size = int(len(X) * 0.8)
X_train0, y_train0 = X[:train_size], y[:train_size]
X_test0, y_test0 = X[train_size:], y[train_size:]

# Standardize the training data
scaler = MinMaxScaler(feature_range=(0, 1))
X_train= scaler.fit_transform(X_train0.reshape(-1, 1)).reshape(X_train0.shape)
y_train = scaler.fit_transform(y_train0.reshape(-1, 1)).reshape(-1)

# Apply the same scaling to the test data
X_test = scaler.transform(X_test0.reshape(-1, 1)).reshape(X_test0.shape)
y_test = scaler.transform(y_test0.reshape(-1, 1)).reshape(-1)



# Instantiate and fit the SVR model with initial parameters
svr_rbf = SVR(kernel='rbf')

# Use GridSearchCV or a similar method to adaptively find better hyperparameters based on initial performance
parameters = {'C': [1e-1, 1e0, 1e1, 1e2], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4], 'epsilon': [1e-1, 1e-2, 1e-3, 1e-4]}
grid_search = GridSearchCV(svr_rbf, parameters, cv=5)
grid_search.fit(np.squeeze(X_train), y_train.ravel())

best_svr = grid_search.best_estimator_

# Predictions with the optimized model
train_predict = best_svr.predict(np.squeeze(X_train))
test_predict = best_svr.predict(np.squeeze(X_test))

# Inverse transform the predictions back to original scale
train_predict = scaler.inverse_transform(train_predict.reshape(-1, 1))
test_predict = scaler.inverse_transform(test_predict.reshape(-1, 1))
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Evaluate the model using MAE, RMSE, MAPE, and Directional Accuracy
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_directional_accuracy(y_true, y_pred):
    correct = np.sum(np.sign(y_true) == np.sign(y_pred))
    return correct / len(y_true)

mae_train = mean_absolute_error(y_train_inv, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train_inv, train_predict))
mape_train = calculate_mape(y_train_inv, train_predict)
direction_train = calculate_directional_accuracy(y_train_inv, train_predict)

mae_test = mean_absolute_error(y_test_inv, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test_inv, test_predict))
mape_test = calculate_mape(y_test_inv, test_predict)
direction_test = calculate_directional_accuracy(y_test_inv, test_predict)

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}, MAPE: {mape_train}%, Directional Accuracy: {direction_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}, MAPE: {mape_test}%, Directional Accuracy: {direction_test}")

In [ ]:
#SVR  带有数据增强
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr
from GANg import GANtest
from sklearn.neighbors import KNeighborsRegressor
# Set random seed for reproducibility
from tensorflow import keras
np.random.seed(42)


data_path = 'golddemandQ.csv'  # Update this to the correct path where IBM.csv is located
data = pd.read_csv(data_path)
data=data.iloc[:,0:2]
data.set_index('time', inplace=True)


def create_dataset(data, time_step=2):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

# Create dataset
X, y = create_dataset(data[['closep']].values, time_step)

# Split data into training and testing sets (80% for training, 20% for testing)
train_size = int(len(X) * 0.8)
X_train0, y_train0 = X[:train_size], y[:train_size]
X_test0, y_test0 = X[train_size:], y[train_size:]

# Standardize the training data
scaler = MinMaxScaler(feature_range=(0, 1))
X_train0= scaler.fit_transform(X_train0.reshape(-1, 1)).reshape(X_train0.shape)
y_train0 = scaler.fit_transform(y_train0.reshape(-1, 1)).reshape(-1)

# Apply the same scaling to the test data
X_test = scaler.transform(X_test0.reshape(-1, 1)).reshape(X_test0.shape)
y_test = scaler.transform(y_test0.reshape(-1, 1)).reshape(-1)

'''data augmentation'''
X_train0=pd.DataFrame(X_train0.reshape(-1,2))
y_train0=pd.DataFrame(y_train0.reshape(-1,1))
gen=GANtest(X_train0)
gen.columns = X_train0.columns


'''LRmodel prediction target'''
from sklearn.linear_model import LinearRegression
best_svr0 = LinearRegression()
best_svr0.fit(X_train0, y_train0)


# Predictions with the optimized model
train_predict0 = best_svr0.predict(gen)
# train_predict0.columns = y_train0.columns
train_predict0=pd.DataFrame(train_predict0)
y_train=pd.concat([y_train0,train_predict0],axis=0)
X_train=pd.concat([X_train0,gen],axis=0)


# Instantiate and fit the SVR model with initial parameters
# svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
# # Use GridSearchCV or a similar method to adaptively find better hyperparameters based on initial performance
# parameters = {'C': [1e-1, 1e0, 1e1, 1e2], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4], 'epsilon': [1e-1, 1e-2, 1e-3, 1e-4]}
# grid_search = GridSearchCV(svr_rbf, parameters, cv=5)
# grid_search.fit(X_train, y_train)
# best_svr = grid_search.best_estimator_

best_svr=BaggingRegressor(random_state=42)
# best_svr=BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100)
best_svr.fit(X_train, y_train)
# Predictions with the optimized model
train_predict = best_svr.predict(np.squeeze(X_train))
test_predict = best_svr.predict(np.squeeze(X_test))

# Inverse transform the predictions back to original scale
train_predict = scaler.inverse_transform(train_predict.reshape(-1, 1))
test_predict = scaler.inverse_transform(test_predict.reshape(-1, 1))
y_train_inv = scaler.inverse_transform(np.array(y_train).reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Evaluate the model using MAE, RMSE, MAPE, and Directional Accuracy
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_directional_accuracy(y_true, y_pred):
    correct = np.sum(np.sign(y_true) == np.sign(y_pred))
    return correct / len(y_true)

mae_train = mean_absolute_error(y_train_inv, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train_inv, train_predict))
mape_train = calculate_mape(y_train_inv, train_predict)
direction_train = calculate_directional_accuracy(y_train_inv, train_predict)

mae_test = mean_absolute_error(y_test_inv, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test_inv, test_predict))
mape_test = calculate_mape(y_test_inv, test_predict)
direction_test = calculate_directional_accuracy(y_test_inv, test_predict)

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}, MAPE: {mape_train}%, Directional Accuracy: {direction_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}, MAPE: {mape_test}%, Directional Accuracy: {direction_test}")

In [ ]:

#LSTM + data augmentation
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from GANg import GANtest
with pd.ExcelWriter('LSTMdataaugmentation-demand-mult_sheets.xlsx') as writer:
    for r in range(10):  # 
        # Set random seed for reproducibility
        np.random.seed(42)

        # Load the data
        data_path = 'golddemandQ.csv'  # Update this to the correct path where IBM.csv is located
        data = pd.read_csv(data_path)
        data=data.iloc[:,[0,2]]

        def create_dataset(data, time_step=2):
            X, y = [], []
            for i in range(len(data) - time_step):
                X.append(data[i:i + time_step])
                y.append(data[i + time_step])
            return np.array(X), np.array(y)

        # Create dataset
        X, y = create_dataset(data[['closed']].values, time_step)

        # Split data into training and testing sets (80% for training, 20% for testing)
        train_size = int(len(X) * 0.8)
        X_train0, y_train0 = X[:train_size], y[:train_size]
        X_test0, y_test0 = X[train_size:], y[train_size:]

        # Standardize the training data
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train0= scaler.fit_transform(X_train0.reshape(-1, 1)).reshape(X_train0.shape)
        y_train0 = scaler.fit_transform(y_train0.reshape(-1, 1)).reshape(-1)

        # Apply the same scaling to the test data
        X_test = scaler.transform(X_test0.reshape(-1, 1)).reshape(X_test0.shape)
        y_test = scaler.transform(y_test0.reshape(-1, 1)).reshape(-1)


        ''' data augmentation'''
        X_train0=pd.DataFrame(X_train0.reshape(-1,2))
        y_train0=pd.DataFrame(y_train0.reshape(-1,1))
        gen=GANtest(X_train0)
        gen.columns = X_train0.columns
        '''LR'''
        from sklearn.linear_model import LinearRegression
        best_svr0 = LinearRegression()
        best_svr0.fit(X_train0, y_train0)
 
        # Predictions with the optimized model
        train_predict0 = best_svr0.predict(gen)
        # train_predict0.columns = y_train0.columns
        train_predict0=pd.DataFrame(train_predict0)
        y_train=pd.concat([y_train0,train_predict0],axis=0)
        X_train=pd.concat([X_train0,gen],axis=0)
        X_train= np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1)
        y_train =np.array(y_train).reshape(-1)


        # Build the LSTM model
        model = Sequential()
        model.add(LSTM(128, input_shape=(sequence_length, 1)))
        model.add(Dropout(0.2))
        model.add(Dense(steps))  # The output layer should have 'steps' units if predicting multiple steps

        # Compile the model
        model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

        # Train the model
        model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1, validation_data=(X_test, y_test))

        # Predicting
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Inverse transform the predictions back to original scale
        train_predict = scaler.inverse_transform(train_predict)
        test_predict = scaler.inverse_transform(test_predict)
        y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
        y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Evaluate the model using MAE, RMSE, MAPE, and Directional Accuracy
        def calculate_mape(y_true, y_pred):
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        def calculate_directional_accuracy(y_true, y_pred):
            correct = np.sum(np.sign(y_true) == np.sign(y_pred))
            return correct / len(y_true)

        mae_train = mean_absolute_error(y_train_inv, train_predict)
        rmse_train = np.sqrt(mean_squared_error(y_train_inv, train_predict))
        mape_train = calculate_mape(y_train_inv, train_predict)
        direction_train = calculate_directional_accuracy(y_train_inv, train_predict)

        mae_test = mean_absolute_error(y_test_inv, test_predict)
        rmse_test = np.sqrt(mean_squared_error(y_test_inv, test_predict))
        mape_test = calculate_mape(y_test_inv, test_predict)
        direction_test = calculate_directional_accuracy(y_test_inv, test_predict)

        print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}, MAPE: {mape_train}%, Directional Accuracy: {direction_train}")
        print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}, MAPE: {mape_test}%, Directional Accuracy: {direction_test}")

        metrics1 = {
                'RMSE': rmse_test,
                # 'MSE': mse,
                'MAE': mae_test,
                'MAPE':mape_test,
                # 'Directional Accuracy': direction_accuracy,
                # 'actual_price':y_test_actual,
                # 'predictions': transformer_predictions
            }
        # print(metrics1)
        metrics2={'actual_price':np.squeeze(y_test_inv),
                'predictions': np.squeeze(test_predict)}
        
        # df_metrics = pd.DataFrame([metrics],index=[0])
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        df_metrics1 = pd.DataFrame(metrics1,index=[0])
        df_metrics2 = pd.DataFrame(metrics2)
        df_metrics=pd.concat([df_metrics1,df_metrics2])
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}')

In [ ]:

!pip install prophet
!pip install pystan = 2.19.1.1


In [ ]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error

# Load the data
data_path = 'goldpd.xlsx'  # Update this to the correct path where IBM.csv is located
data = pd.read_excel(data_path)
data=data.iloc[:,0:2]
data['time'] = pd.to_datetime(data['time']).dt.date
valid=data[int(0.8*len(data)):]
train=data[:int(0.8*len(data))]
train_prophet = pd.DataFrame()
train_prophet['ds'] = train.time
train_prophet['y'] = train.closep


#instantiate Prophet with only yearly seasonality as our data is monthly 
# model = Prophet( yearly_seasonality=True)
model = Prophet()
model.fit(train_prophet) #fit the model with your dataframe
# predict for five months in the furure and MS - month start is the frequency
future = model.make_future_dataframe(periods = len(valid), freq = 'Q') 
future.tail()


# now lets make the forecasts
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

fig = model.plot(forecast)
#plot the predictions for validation set
plt.plot(valid['time'],valid['closep'], label='Valid', color = 'red', linewidth = 2)
plt.show()

y_prophet = pd.DataFrame()
y_prophet['ds'] = valid.time
y_prophet['y'] = valid.closep

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_forecast(y,pred):
    results = pd.DataFrame({'r2_score':r2_score(y, pred),
                           }, index=[0])
    results['mean_absolute_error'] = mean_absolute_error(y, pred)
    results['median_absolute_error'] = median_absolute_error(y, pred)
    results['mse'] = mean_squared_error(y, pred)
    results['msle'] = mean_squared_log_error(y, pred)
    results['mape'] = mean_absolute_percentage_error(y, pred)
    results['rmse'] = np.sqrt(results['mse'])
    return results

y_prophet = y_prophet.set_index('ds')
forecast_prophet = forecast.set_index('ds')

evaluate_forecast(y_prophet.y, forecast_prophet.yhat[int(0.8*len(data)):])

In [ ]:
#SVR  month data forecasting 3 steps=quarterly
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr

# Set random seed for reproducibility
np.random.seed(42)

data_path = 'goldm.csv'  # Update this to the correct path where IBM.csv is located
data = pd.read_csv(data_path)
data=data.iloc[:,0:2]
data.set_index('time', inplace=True)

# Assuming 'Close' is the column you want to predict
# data = data['closep'].values.reshape(-1, 1)

# # Normalize the data
# scaler = MinMaxScaler(feature_range=(-1, 1))
# normalized_data = scaler.fit_transform(data)

# # Data Preprocessing Function modified for multi-step
# def create_sequences(data, sequence_length, steps):
#     xs, ys = [], []
#     for i in range(len(data) - sequence_length - steps + 1):
#         x = data[i:(i + sequence_length)]
#         y = data[i + sequence_length:i + sequence_length + steps]
#         xs.append(x)
#         ys.append(y)
#     return np.array(xs), np.array(ys)

# # Create sequences with modified function
# sequence_length = 2
# steps = 1  # Number of future steps to predict
# X, y = create_sequences(normalized_data, sequence_length, steps)

# # Split the data
# split = int(len(X) * 0.8)
# X_train, X_test = X[:split], X[split:]
# y_train, y_test = y[:split], y[split:]
time_step=2
ste=3
def create_dataset(data, time_step=2, ste=2):
    X, y = [], []
    for i in range(len(data) - time_step-ste+1):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step+ste-1])
    return np.array(X), np.array(y)

# Create dataset
X, y = create_dataset(data[['close']].values, time_step)

# Split data into training and testing sets (80% for training, 20% for testing)
train_size = int(len(X) * 0.94)
X_train0, y_train0 = X[:train_size], y[:train_size]
X_test0, y_test0 = X[train_size:], y[train_size:]

# Standardize the training data
scaler = MinMaxScaler(feature_range=(0, 1))
X_train= scaler.fit_transform(X_train0.reshape(-1, 1)).reshape(X_train0.shape)
y_train = scaler.fit_transform(y_train0.reshape(-1, 1)).reshape(-1)

# Apply the same scaling to the test data
X_test = scaler.transform(X_test0.reshape(-1, 1)).reshape(X_test0.shape)
y_test = scaler.transform(y_test0.reshape(-1, 1)).reshape(-1)



# Instantiate and fit the SVR model with initial parameters
# svr_rbf = SVR(kernel='rbf')
# # Use GridSearchCV or a similar method to adaptively find better hyperparameters based on initial performance
# parameters = {'C': [1e-1, 1e0, 1e1, 1e2], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4], 'epsilon': [1e-1, 1e-2, 1e-3, 1e-4]}
# grid_search = GridSearchCV(svr_rbf, parameters, cv=5)
# grid_search.fit(np.squeeze(X_train), y_train.ravel())
# best_svr = grid_search.best_estimator_

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
# best_svr=BaggingRegressor()
# best_svr=BaggingRegressor(random_state=42)
best_svr=BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100)
best_svr.fit(np.squeeze(X_train), y_train.ravel())



# Predictions with the optimized model
train_predict = best_svr.predict(np.squeeze(X_train))
test_predict = best_svr.predict(np.squeeze(X_test))

# Inverse transform the predictions back to original scale
train_predict = scaler.inverse_transform(train_predict.reshape(-1, 1))
test_predict = scaler.inverse_transform(test_predict.reshape(-1, 1))
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Evaluate the model using MAE, RMSE, MAPE, and Directional Accuracy
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_directional_accuracy(y_true, y_pred):
    correct = np.sum(np.sign(y_true) == np.sign(y_pred))
    return correct / len(y_true)

mae_train = mean_absolute_error(y_train_inv, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train_inv, train_predict))
mape_train = calculate_mape(y_train_inv, train_predict)
direction_train = calculate_directional_accuracy(y_train_inv, train_predict)

mae_test = mean_absolute_error(y_test_inv, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test_inv, test_predict))
mape_test = calculate_mape(y_test_inv, test_predict)
direction_test = calculate_directional_accuracy(y_test_inv, test_predict)

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}, MAPE: {mape_train}%, Directional Accuracy: {direction_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}, MAPE: {mape_test}%, Directional Accuracy: {direction_test}")


#high frequency tranform low frequency
summed_data_true= y_test_inv.reshape(-1, 3).mean(axis=1)
summed_data_predict=test_predict.reshape(-1, 3).mean(axis=1)
pd.concat([pd.DataFrame(summed_data_true),pd.DataFrame(summed_data_predict)],axis=1).to_csv('yuedujieguo1.csv')
mae_te = mean_absolute_error(summed_data_true, summed_data_predict)
rmse_te = np.sqrt(mean_squared_error(summed_data_true, summed_data_predict))
mape_te = calculate_mape(summed_data_true, summed_data_predict)
print(f"final Testing Data - MAE: {mae_te}, RMSE: {rmse_te}, MAPE: {mape_te}%")


In [ ]:

#LSTM + month data forecasting
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from GANg import GANtest
with pd.ExcelWriter('LSTMhighfrequency-price-mult_sheets.xlsx') as writer:
    for r in range(10):  
        # Set random seed for reproducibility
        np.random.seed(42)

        # Load the data
        data_path = 'goldm.csv'  # Update this to the correct path where IBM.csv is located
        data = pd.read_csv(data_path)
        data=data.iloc[:,0:2]
        data.set_index('time', inplace=True)

  
        sequence_length = 2  # Updated sequence length for your use case
        steps = 1  # Number of future steps to predict
 

        time_step=2
        ste=3
        def create_dataset(data, time_step=2, ste=2):
            X, y = [], []
            for i in range(len(data) - time_step-ste+1):
                X.append(data[i:i + time_step])
                y.append(data[i + time_step+ste-1])
            return np.array(X), np.array(y)

        # Create dataset
        X, y = create_dataset(data[['close']].values, time_step)

        # Split data into training and testing sets (80% for training, 20% for testing)
        train_size = int(len(X) * 0.94)
        X_train0, y_train0 = X[:train_size], y[:train_size]
        X_test0, y_test0 = X[train_size:], y[train_size:]

        # Standardize the training data
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train0= scaler.fit_transform(X_train0.reshape(-1, 1)).reshape(X_train0.shape)
        y_train0 = scaler.fit_transform(y_train0.reshape(-1, 1)).reshape(-1)

        # Apply the same scaling to the test data
        X_test = scaler.transform(X_test0.reshape(-1, 1)).reshape(X_test0.shape)
        y_test = scaler.transform(y_test0.reshape(-1, 1)).reshape(-1)


        # Build the LSTM model
        model = Sequential()
        model.add(LSTM(128, input_shape=(sequence_length, 1)))
        model.add(Dropout(0.2))
        model.add(Dense(steps))  # The output layer should have 'steps' units if predicting multiple steps

        # Compile the model
        model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

        # Train the model
        model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1, validation_data=(X_test, y_test))

        # Predicting
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Inverse transform the predictions back to original scale
        train_predict = scaler.inverse_transform(train_predict)
        test_predict = scaler.inverse_transform(test_predict)
        y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
        y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Evaluate the model using MAE, RMSE, MAPE, and Directional Accuracy
        def calculate_mape(y_true, y_pred):
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        def calculate_directional_accuracy(y_true, y_pred):
            correct = np.sum(np.sign(y_true) == np.sign(y_pred))
            return correct / len(y_true)

        mae_train = mean_absolute_error(y_train_inv, train_predict)
        rmse_train = np.sqrt(mean_squared_error(y_train_inv, train_predict))
        mape_train = calculate_mape(y_train_inv, train_predict)
        direction_train = calculate_directional_accuracy(y_train_inv, train_predict)

        mae_test = mean_absolute_error(y_test_inv, test_predict)
        rmse_test = np.sqrt(mean_squared_error(y_test_inv, test_predict))
        mape_test = calculate_mape(y_test_inv, test_predict)
        direction_test = calculate_directional_accuracy(y_test_inv, test_predict)

        print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}, MAPE: {mape_train}%, Directional Accuracy: {direction_train}")
        print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}, MAPE: {mape_test}%, Directional Accuracy: {direction_test}")



        summed_data_true= y_test_inv.reshape(-1, 3).mean(axis=1)
        summed_data_predict=test_predict.reshape(-1, 3).mean(axis=1)
        # pd.concat([pd.DataFrame(summed_data_true),pd.DataFrame(summed_data_predict)],axis=1).to_csv('yuedujieguo1.csv')
        mae_te = mean_absolute_error(summed_data_true, summed_data_predict)
        rmse_te = np.sqrt(mean_squared_error(summed_data_true, summed_data_predict))
        mape_te = calculate_mape(summed_data_true, summed_data_predict)
        print(f"final Testing Data - MAE: {mae_te}, RMSE: {rmse_te}, MAPE: {mape_te}%")


        metrics1 = {
                'RMSE': rmse_te,
                # 'MSE': mse,
                'MAE': mae_te,
                'MAPE':mape_te,
                # 'Directional Accuracy': direction_accuracy,
                # 'actual_price':y_test_actual,
                # 'predictions': transformer_predictions
            }
        # print(metrics1)
        metrics2={'actual_price':np.squeeze(summed_data_true),
                'predictions': np.squeeze(summed_data_predict)}
   
        # df_metrics = pd.DataFrame([metrics],index=[0])
        # metrics.to_excel('model_metrics.xlsx', index=metrics)
        df_metrics1 = pd.DataFrame(metrics1,index=[0])
        df_metrics2 = pd.DataFrame(metrics2)
        df_metrics=pd.concat([df_metrics1,df_metrics2])
        # df_metrics.to_excel('model_metrics_random_forest84.xlsx', index=metrics)
        df_metrics.to_excel(writer, sheet_name=f'Run_{r}')

multivariables forecasting

In [ ]:
#SVR
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr

# Set random seed for reproducibility
np.random.seed(42)

# data_path = 'goldmultivariables.xlsx'  # Update this to the correct path where IBM.csv is located
data_path = 'golddemanmultivariables.xlsx'  # Update this to the correct path where IBM.csv is located
data = pd.read_excel(data_path)
data=data.iloc[:,:]
data.set_index('time', inplace=True)

train_size = int(len(data) * 0.8)
X_train, y_train = data.iloc[:train_size,1:], data.iloc[:train_size,0]
X_test, y_test = data.iloc[train_size:,1:], data.iloc[train_size:,0]

# Standardize the training data
scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler()


X_train = scaler_x.fit_transform(X_train)
X_test= scaler_x.transform(X_test)


# scalers = {}
# for i in range(X_train.shape[1]):
#     scalers[i] = MinMaxScaler(feature_range=(0, 1))
#     X_train.iloc[:, i] = scalers[i].fit_transform(X_train.iloc[:, i].values.reshape(-1, 1))
# for i in range(X_test.shape[1]):
#     X_test.iloc[:, i] = scalers[i].transform(X_test.iloc[:, i].values.reshape(-1, 1))

y_train = scaler_y.fit_transform(np.array(y_train).reshape(-1, 1)).flatten()
y_test = scaler_y.transform(np.array(y_test).reshape(-1, 1)).flatten()

# Instantiate and fit the SVR model with initial parameters
# svr_rbf = SVR(kernel='rbf')
# # Use GridSearchCV or a similar method to adaptively find better hyperparameters based on initial performance
# parameters = {'C': [1e-1, 1e0, 1e1, 1e2], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4], 'epsilon': [1e-1, 1e-2, 1e-3, 1e-4]}
# grid_search = GridSearchCV(svr_rbf, parameters, cv=5)
# grid_search.fit(np.squeeze(X_train), y_train.ravel())
# best_svr = grid_search.best_estimator_


from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
# best_svr=BaggingRegressor()
# best_svr=BaggingRegressor(random_state=42)
best_svr=BaggingRegressor(estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=100)
best_svr.fit(np.squeeze(X_train), y_train.ravel())


# Predictions with the optimized model
train_predict = best_svr.predict(np.squeeze(X_train))
test_predict = best_svr.predict(np.squeeze(X_test))

# Inverse transform the predictions back to original scale
# train_predict = scaler.inverse_transform(train_predict.reshape(-1, 1))
# test_predict = scaler.inverse_transform(test_predict.reshape(-1, 1))
# y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
# y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()
test_predict = scaler_y.inverse_transform(test_predict.reshape(-1, 1)).flatten()
# Evaluate the model using MAE, RMSE, MAPE, and Directional Accuracy
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_directional_accuracy(y_true, y_pred):
    correct = np.sum(np.sign(y_true) == np.sign(y_pred))
    return correct / len(y_true)



mae_test = mean_absolute_error(y_test_inv, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test_inv, test_predict))
mape_test = calculate_mape(y_test_inv, test_predict)
direction_test = calculate_directional_accuracy(y_test_inv, test_predict)


print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}, MAPE: {mape_test}%, Directional Accuracy: {direction_test}")